In [1]:
import pandas as pd

file_path = r"C:\Users\LENOVO\OneDrive\AICTE\AI Medical Diagnosis System\Dataset Text Based\prepocessed_lungs_data.csv"
df = pd.read_csv(file_path)
print(df.info())
print(df.head())

print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   Unnamed: 0             309 non-null    int64
 1   GENDER                 309 non-null    int64
 2   AGE                    309 non-null    int64
 3   SMOKING                309 non-null    int64
 4   YELLOW_FINGERS         309 non-null    int64
 5   ANXIETY                309 non-null    int64
 6   PEER_PRESSURE          309 non-null    int64
 7   CHRONIC DISEASE        309 non-null    int64
 8   FATIGUE                309 non-null    int64
 9   ALLERGY                309 non-null    int64
 10  WHEEZING               309 non-null    int64
 11  ALCOHOL CONSUMING      309 non-null    int64
 12  COUGHING               309 non-null    int64
 13  SHORTNESS OF BREATH    309 non-null    int64
 14  SWALLOWING DIFFICULTY  309 non-null    int64
 15  CHEST PAIN             309 non-null    i

In [3]:
print(df.columns)

Index(['Unnamed: 0', 'GENDER', 'AGE', 'SMOKING', 'YELLOW_FINGERS', 'ANXIETY',
       'PEER_PRESSURE', 'CHRONIC DISEASE', 'FATIGUE ', 'ALLERGY ', 'WHEEZING',
       'ALCOHOL CONSUMING', 'COUGHING', 'SHORTNESS OF BREATH',
       'SWALLOWING DIFFICULTY', 'CHEST PAIN', 'LUNG_CANCER'],
      dtype='object')


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np

# Specify the target column
target_col = "SWALLOWING DIFFICULTY"

# Handle missing values using median imputation
imputer = SimpleImputer(strategy="median")
df[df.columns] = imputer.fit_transform(df)

# Identify categorical columns (excluding the target)
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
if target_col in cat_cols:
    cat_cols.remove(target_col)

# Encode categorical columns
if cat_cols:
    encoder = OrdinalEncoder()
    df[cat_cols] = encoder.fit_transform(df[cat_cols])

# Encode the target column
target_encoder = LabelEncoder()
df[target_col] = target_encoder.fit_transform(df[target_col])

# Separate features and target
X = df.drop(columns=[target_col])
y = df[target_col]

# Standardize only numerical features
num_cols = X.select_dtypes(include=["number"]).columns
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

# Convert back to DataFrame (optional)
X = pd.DataFrame(X, columns=df.drop(columns=[target_col]).columns)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("✅ Data Preprocessing Complete!")


✅ Data Preprocessing Complete!


In [7]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load Dataset (Ensure df is available)
try:
    df  # Check if df is already loaded
except NameError:
    raise ValueError("Dataset 'df' is not defined. Load your dataset before running this code.")

# Clean Column Names (Fix JSON Errors in CatBoost)
df.columns = df.columns.str.replace(r"[^a-zA-Z0-9_]", "_", regex=True)

# Specify Target Columns
target_cols = ["SWALLOWING_DIFFICULTY", "YELLOW_FINGERS"]  # Adjust as per dataset

# Handle Missing Values (Median Imputation)
imputer = SimpleImputer(strategy="median")
df[df.columns] = imputer.fit_transform(df)

# Identify Categorical Columns (excluding target columns)
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
cat_cols = [col for col in cat_cols if col not in target_cols]

# Encode Categorical Features
if cat_cols:
    encoder = OrdinalEncoder()
    df[cat_cols] = encoder.fit_transform(df[cat_cols])

# Encode Target Columns
label_encoders = {}
for target_col in target_cols:
    label_encoders[target_col] = LabelEncoder()
    df[target_col] = label_encoders[target_col].fit_transform(df[target_col])

# Separate Features and Targets
X = df.drop(columns=target_cols)
y = df[target_cols]  # Multi-label classification

# Standardize Only Numerical Features
num_cols = X.select_dtypes(include=["number"]).columns
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

# Convert Back to DataFrame
X = pd.DataFrame(X, columns=df.drop(columns=target_cols).columns)

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y[target_cols[0]], test_size=0.2, random_state=42, stratify=y[target_cols[0]])

# Train Models (Without LGBM)
models = {
    "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42),  # Removed use_label_encoder
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
}

# Train & Evaluate Each Model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\n{name} Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"{name} Classification Report:\n{classification_report(y_test, y_pred)}")

print("\n✅ Data Preprocessing & Model Training Complete!")



XGBoost Accuracy: 0.8226
XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.91      0.85        33
           1       0.88      0.72      0.79        29

    accuracy                           0.82        62
   macro avg       0.83      0.82      0.82        62
weighted avg       0.83      0.82      0.82        62


CatBoost Accuracy: 0.9032
CatBoost Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.97      0.91        33
           1       0.96      0.83      0.89        29

    accuracy                           0.90        62
   macro avg       0.91      0.90      0.90        62
weighted avg       0.91      0.90      0.90        62


Gradient Boosting Accuracy: 0.8387
Gradient Boosting Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.91      0.86        33
           1       0.88      0.76      0.81 

In [8]:
from lightgbm import LGBMClassifier
import joblib

# Train the model (example)
best_lgbm = LGBMClassifier()
best_lgbm.fit(X_train, y_train)  # Ensure X_train and y_train are defined

# Save the trained model
model_filename = "Lungs_model.sav"
joblib.dump(best_lgbm, model_filename)
print(f"Model saved as {model_filename}")

[LightGBM] [Info] Number of positive: 116, number of negative: 131
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000277 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 154
[LightGBM] [Info] Number of data points in the train set: 247, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.469636 -> initscore=-0.121607
[LightGBM] [Info] Start training from score -0.121607
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best